# LlamaIndex와 AgentCore Memory - 법률 문서 분석기 (장기 메모리)

## 소개

이 노트북은 Amazon Bedrock AgentCore Memory 기능을 LlamaIndex와 통합하여 **장기 메모리** 지속성을 갖춘 법률 문서 분석기를 만드는 방법을 보여줍니다. 여러 사건과 법적 절차에 걸쳐 누적된 법률 지식을 구축하고 수개월 및 수년에 걸쳐 판례를 추적할 수 있습니다.

## 아키텍처 개요

![LlamaIndex AgentCore Long-Term Memory Architecture](LlamaIndex-AgentCore-LTM-Arch.png)

## 튜토리얼 세부사항

**튜토리얼 세부사항:**
- **튜토리얼 유형**: 장기 세션 간 메모리
- **Agent 사용 사례**: 법률 문서 분석기
- **Agentic Framework**: LlamaIndex
- **LLM model**: Anthropic Claude 3.7 Sonnet
- **튜토리얼 구성 요소**: AgentCore 장기 메모리, LlamaIndex Agent, 법률 Tool
- **예제 복잡도**: 고급

## 비즈니스 가치

**엔터프라이즈 법률 인텔리전스**: 사건 지식을 축적하고, 법률 전략 진화를 추적하며, 사건과 기간에 걸쳐 포괄적인 판례 데이터베이스를 유지하는 지속적인 AI 메모리로 법률 실무를 혁신하세요.

**주요 전문적 이점:**
- **사건 연속성**: 법률 사안과 팀 구성원 간의 원활한 지식 전달
- **판례 데이터베이스**: 중요한 판례법, 전략 및 결과를 영구적으로 보존
- **사건 간 인텔리전스**: 여러 법률 사안에 걸친 패턴과 연결 식별
- **전략적 우위**: 우수한 법률 포지셔닝을 위한 과거 사건 데이터 활용
- **고객 가치**: 다년간의 고객 관계에 대한 상세한 컨텍스트 유지
- **리스크 관리**: 규제 변경 사항과 법률 전략에 미치는 영향 추적

## 장기 메모리 구성

**기술 설정**: 이 튜토리얼은 12개월 보존 기간을 가진 Semantic Strategy와 함께 AgentCore Memory를 사용합니다:
- **메모리 유형**: 자동 인사이트 추출을 사용하는 Semantic strategy
- **보존 기간**: 법률 사건 연속성을 위한 365일 이벤트 만료
- **세션 간**: 동일한 actor_id + memory_id, 법률 기간별로 다른 session_id
- **검색 기능**: 사건 이력 전반에 걸친 의미론적 검색을 위한 내장 메모리 검색 도구

## 기술 개요

**주요 장기 메모리 구성 요소:**
1. **Semantic Strategy 구성**: 365일 보존 기간을 가진 자동 인사이트 추출을 위해 SemanticStrategy 사용
2. **세션 간 지속성**: 동일한 actor_id + memory_id, 기간별로 다른 session_id로 지식 연속성 활성화
3. **커스텀 메모리 검색 Tool**: AgentCore의 네이티브 search_long_term_memories()를 LlamaIndex FunctionTool로 래핑
4. **의미론적 처리 파이프라인**: 대화 이벤트 → 의미론적 메모리 변환을 위한 90초 대기
5. **동적 세션 관리**: 유연한 세션 처리를 위해 memory.context.session_id 사용

**학습 내용:**

- 여러 법률 사건에 걸쳐 지속적인 AgentCore Memory 생성
- 시간이 지남에 따라 누적되는 법률 지식 구축
- 판례법과 판례 전반에 걸친 의미론적 검색 구현
- 법률 전략 진화 및 사건 결과 추적
- 세션 간 법률 지식 지속성 및 검색 테스트

## 시나리오 컨텍스트

이 예제에서는 수개월 및 수년에 걸친 여러 사건에 걸쳐 법률 지식을 유지하는 "법률 문서 분석기"를 만듭니다. 분석기는 AgentCore Memory를 사용하여 시간이 지남에 따라 성장하고 진화하는 판례, 법률 전략, 규제 변경 사항 및 사건 결과의 지속적인 지식 베이스를 구축하여 정교한 종단 법률 분석을 가능하게 합니다.

## 사전 요구 사항

- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory 권한이 있는 AWS IAM 역할:
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock model에 대한 액세스

## 단계 1: 종속성 설치 및 설정

In [ ]:
# semantic strategy toolkit 포함한 필수 라이브러리 설치
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3 bedrock-agentcore-starter-toolkit

In [ ]:
# 필수 컴포넌트 import
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.session import MemorySessionManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies.semantic import SemanticStrategy
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from datetime import datetime
import os

print("✅ All dependencies imported successfully!")

## 단계 2: AgentCore Memory 구성

장기 법률 지식을 위한 AgentCore Memory 리소스를 생성하거나 가져옵니다:

In [ ]:
# Semantic Strategy를 사용하여 AgentCore Memory 생성
region = os.getenv('AWS_REGION', 'us-east-1')
memory_manager = MemoryManager(region_name=region)

try:
    # 자동 인사이트 추출을 위한 semantic strategy로 메모리 생성
    memory = memory_manager.get_or_create_memory(
        name=f'LegalAnalyzerSemantic_{int(datetime.now().timestamp())}',
        strategies=[SemanticStrategy(name="legalLongTermMemory")],
        event_expiry_days=365  # 법률 기록을 위한 12개월 보존
    )
    memory_id = memory.get('id')
    print(f"✅ Created Semantic Memory: {memory_id}")
    print(f"   Status: {memory.get('status')}")
    print(f"   Strategies: {[s.get('name') if isinstance(s, dict) else str(s) for s in memory.get('strategies', [])]}")
    
    # 메모리가 ACTIVE 상태가 될 때까지 대기
    if memory.get('status') != 'ACTIVE':
        print(f"\n⏳ Waiting for memory to become ACTIVE (currently {memory.get('status')})...")
        import time
        max_wait = 300  # 최대 5분
        waited = 0
        while waited < max_wait:
            time.sleep(10)
            waited += 10
            # 상태 확인
            current_memory = memory_manager.get_memory(memory_id)
            status = current_memory.get('status')
            print(f"   [{waited}s] Status: {status}")
            if status == 'ACTIVE':
                print(f"✅ Memory is now ACTIVE! (took {waited} seconds)")
                break
        else:
            print(f"⚠️  Memory still not ACTIVE after {max_wait}s. Proceeding anyway...")
    
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # 기존 memory ID로 교체

## 단계 3: 법률 Tool 구현

종단 법률 분석을 위한 전문 도구를 정의합니다:

In [ ]:
def analyze_contract_clause(case_id: str, clause_type: str, risk_level: str, precedent_reference: str) -> str:
    """위험 평가 및 판례 참조를 포함한 계약 조항 분석"""
    return f"📋 Analyzed {clause_type} clause for {case_id} (Risk: {risk_level})"

def track_case_precedent(case_id: str, precedent_case: str, legal_principle: str, applicability: str) -> str:
    """법적 원칙 및 적용 가능성 평가를 포함한 사건 판례 추적"""
    return f"⚖️ {case_id} precedent: {precedent_case} - {legal_principle}"

def document_regulatory_change(regulation_type: str, change_description: str, impact_assessment: str, client_implications: str) -> str:
    """고객 포트폴리오 영향을 포함한 규제 변경 문서화"""
    print(f"📜 Regulatory update: {regulation_type} - {change_description} (Impact: {impact_assessment})")
    return f"Documented regulatory change: {regulation_type}"

def update_legal_strategy(case_id: str, strategy_type: str, approach: str, confidence_level: str) -> str:
    """특정 사건에 대한 법률 전략 업데이트"""
    print(f"🎯 Legal strategy: {case_id} - {strategy_type} ({confidence_level} confidence)")
    return f"Updated strategy for {case_id}"

def log_case_outcome(case_id: str, outcome_type: str, result: str, lessons_learned: str) -> str:
    """교훈과 함께 사건 결과 기록"""
    print(f"🏛️ Case outcome: {case_id} - {outcome_type}: {result}")
    return f"Logged outcome for {case_id}"

def log_legal_milestone(quarter: str, milestone: str, details: str) -> str:
    """분기 및 상세 진행 상황과 함께 법률 마일스톤 기록"""
    print(f"🎯 {quarter} milestone: {milestone}")
    return f"Logged milestone for {quarter}: {milestone} - {details}"

def track_legal_metrics(metric_type: str, value: str, case_id: str, quarter: str) -> str:
    """사건 및 타임라인과 함께 특정 법률 지표 추적"""
    print(f"📊 {quarter}: {metric_type} = {value} (for {case_id})")
    return f"Tracked {metric_type}: {value} for {case_id} in {quarter}"

def save_legal_insight(insight: str, quarter: str, legal_context: str) -> str:
    """컨텍스트와 함께 법률 인사이트 저장"""
    print(f"💡 {quarter} insight: {insight[:50]}...")
    return f"Saved {quarter} insight with legal context: {legal_context}"

# agent를 위한 tool 객체 생성
legal_tools = [
    FunctionTool.from_defaults(fn=analyze_contract_clause),
    FunctionTool.from_defaults(fn=track_case_precedent),
    FunctionTool.from_defaults(fn=document_regulatory_change),
    FunctionTool.from_defaults(fn=update_legal_strategy),
    FunctionTool.from_defaults(fn=log_case_outcome),
    FunctionTool.from_defaults(fn=log_legal_milestone),
    FunctionTool.from_defaults(fn=track_legal_metrics),
    FunctionTool.from_defaults(fn=save_legal_insight)
]

print("✅ Legal tools created!")

## 단계 3b: 메모리 검색 Tool 추가

Agent가 장기 메모리를 검색할 수 있도록 하는 도구를 생성합니다:

In [ ]:
def create_memory_retrieval_tool(memory_id: str, actor_id: str, region: str):
    """agent가 자신의 장기 메모리를 검색할 수 있는 도구 생성"""
    
    def search_long_term_memory(query: str) -> str:
        """사건, 판례, 전략 및 결과에 대한 관련 법률 정보를 장기 메모리에서 검색.
        
        다음을 기억해야 할 때 이 도구 사용:
        - 사건 정보 (판례, 전략, 결과)
        - 법적 판례 및 적용
        - 규제 변경 및 영향
        - 법률 전략 및 효과성
        - 사건 결과 및 교훈
        
        Args:
            query: 필요한 정보를 설명하는 검색 쿼리 (예: 'CASE-001 precedents', 'contract strategies', 'Q1 outcomes')
        
        Returns:
            장기 메모리에서 가져온 관련 정보
        """
        try:
            from bedrock_agentcore.memory.session import MemorySessionManager
            
            # session manager 생성 (memory_id와 region만 필요)
            session_manager = MemorySessionManager(
                memory_id=memory_id,
                region_name=region
            )
            
            # semantic strategy namespace에서 장기 메모리 검색
            results = session_manager.search_long_term_memories(
                query=query,
                namespace_prefix="/strategies/",  # semantic strategy namespace에서 검색
                top_k=5,
                max_results=10
            )
            
            if not results:
                return "No relevant information found in long-term memory. This might be new information or the memory extraction may still be processing."
            
            # agent를 위한 결과 포맷팅
            output = "📚 Retrieved from long-term memory:\\n\\n"
            for i, result in enumerate(results, 1):
                # MemoryRecord 객체 - content 속성 접근
                content = getattr(result, 'content', str(result))
                # 매우 긴 콘텐츠 잘라내기
                if len(content) > 300:
                    content = content[:300] + "..."
                output += f"{i}. {content}\\n\\n"
            
            return output
            
        except Exception as e:
            return f"⚠️ Error searching memory: {str(e)}. Proceeding without historical context."
    
    return FunctionTool.from_defaults(fn=search_long_term_memory)

# 메모리 검색 도구 생성
memory_search_tool = create_memory_retrieval_tool(memory_id, "legal-analyst", region)

# 도구 목록에 메모리 검색 추가
legal_tools_with_memory = legal_tools + [memory_search_tool]

print(f"✅ Memory retrieval tool created! Total tools: {len(legal_tools_with_memory)}")
print("   Using namespace: /strategies/ (for semantic strategy compatibility)")

## 단계 3c: 메모리 구성 확인

Semantic strategy가 올바르게 구성되었는지 확인합니다:

In [ ]:
# 메모리 구성 확인
memory_info = memory_manager.get_memory(memory_id)
print(f"Strategies: {memory_info.get('strategies')}")
print(f"Status: {memory_info.get('status')}")
print(f"Name: {memory_info.get('name')}")

# strategy 세부 정보 표시
strategies = memory_info.get('strategies', [])
for strategy in strategies:
    print(f"\nStrategy Details:")
    print(f"  Name: {strategy.get('name')}")
    print(f"  Type: {strategy.get('type')}")
    print(f"  Status: {strategy.get('status')}")
    print(f"  ID: {strategy.get('strategyId')}")

## 단계 4: 다중 세션 Agent 구현

다양한 법률 기간을 시뮬레이션하는 헬퍼 함수를 생성합니다:

In [ ]:
# 장기 메모리 구성 (세션 간)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
ANALYST_ID = "legal-analyst"  # 모든 세션에서 동일한 analyst

def create_legal_session(session_name: str):
    """장기 메모리 지속성을 가진 새로운 법률 세션 생성"""
    context = AgentCoreMemoryContext(
        actor_id=ANALYST_ID,         # 동일한 analyst
        memory_id=memory_id,         # 동일한 메모리 저장소 (장기 메모리 활성화)
        session_id=f"legal-{session_name}", # 기간별로 다른 세션
        namespace="/legal-analysis/"
    )
    
    memory = AgentCoreMemory(context=context)
    llm = BedrockConverse(model=MODEL_ID)
    agent = FunctionAgent(
        tools=legal_tools_with_memory,  # 메모리 검색 기능이 있는 도구 사용
        llm=llm, 
        verbose=True,  # 메모리 검색 시점을 보기 위해 verbose 활성화
        system_prompt="""You are a senior legal analyst with access to long-term memory.
        
CRITICAL: When asked about cases, precedents, strategies, or historical information, 
you MUST use the search_long_term_memory tool FIRST before responding.

For example:
- "What cases am I analyzing?" → Use search_long_term_memory("cases precedents")
- "What strategies have I used?" → Use search_long_term_memory("legal strategies")
- "What outcomes have I achieved?" → Use search_long_term_memory("case outcomes")

Always provide conclusive, complete responses without asking follow-up questions.\n
Execute all requested actions immediately and completely. Provide detailed, professional responses."""
    )
    
    return agent, memory

print("✅ Multi-session Legal Document Analyzer setup complete!")


## 단계 5: Q1 법률 세션 - 초기 사건 분석

첫 번째 법률 세션을 시작하고 사건 기준선을 설정합니다:

In [ ]:
# === Q1 법률 세션 ===
print("🗓️ === Q1: INITIAL CASE ANALYSIS ===")

agent_q1, memory_q1 = create_legal_session("q1")

# 초기 계약 조항 분석
response = await agent_q1.run(
    "I'm Senior Legal Analyst Sarah Chen. Analyze contract clause for 'CASE-001' with clause type 'Indemnification', "
    "risk level 'High', precedent reference 'Smith v. Johnson (2019) - broad indemnification scope creates "
    "significant liability exposure for client'.",
    memory=memory_q1
)

print("🎯 Q1 Initial Analysis:")
print(response)

In [ ]:
# 초기 법률 전략 문서화
response = await agent_q1.run(
    "Update legal strategy for 'CASE-001': strategy type 'Contract Negotiation', "
    "approach 'narrow indemnification scope, add carve-outs for gross negligence and willful misconduct', confidence level 'high'.",
    memory=memory_q1
)
print("💭 Q1 Contract Strategy:", response)

response = await agent_q1.run(
    "Update legal strategy for 'CASE-001': strategy type 'Risk Mitigation', "
    "approach 'insurance requirements, liability caps, and mutual indemnification structure', confidence level 'medium'.",
    memory=memory_q1
)
print("💭 Q1 Risk Strategy:", response)

In [ ]:
# 이벤트가 저장되었는지 확인
print("\n🔍 Verifying events were stored...")
try:
    from bedrock_agentcore.memory import MemoryClient
    client = MemoryClient(region_name=region)
    # 메모리 컨텍스트에서 session_id 가져오기
    current_session_id = memory_q1.context.session_id
    events = client.list_events(
        memory_id=memory_id,
        actor_id=ANALYST_ID,
        session_id=current_session_id  # 동적 - 현재 세션 사용
    )
    print(f"✅ Stored {len(events)} conversational events in {current_session_id}")
except Exception as e:
    print(f"⚠️  Could not verify events: {e}")

# semantic 메모리 처리를 위한 대기 시간
import asyncio
print("\n⏳ Waiting for semantic memory extraction and indexing...")
print("   (AgentCore processes conversational events in the background)")
await asyncio.sleep(90)  # 메모리 추출을 위한 대기 시간 증가
print("✅ Memory processing complete - memories should now be searchable")

## 단계 6: Q2 법률 세션 - 규제 업데이트 대응

장기 메모리 검색을 테스트하고 규제 변경에 적응합니다:

In [ ]:
# === Q2 법률 세션 ===
print("\n🗓️ === Q2: REGULATORY UPDATE RESPONSE (NEW SESSION) ===")

agent_q2, memory_q2 = create_legal_session("q2")

# 세션 간 사건 회상 테스트 - agent가 search_long_term_memory 도구를 사용해야 함
print("\n🧠 Testing memory retrieval across sessions...")
print("   (Watch for the agent to use search_long_term_memory tool)\n")

response = await agent_q2.run(
    "What cases am I analyzing? What are their risk levels, strategies, and precedents?",
    memory=memory_q2
)

print("\n🧠 Q2 Case Recall:")
print(response)
print("\n✅ Expected: CASE-001, indemnification analysis, Smith v. Johnson precedent")

In [ ]:
# 규제 변경 문서화
response = await agent_q2.run(
    "Document regulatory change: regulation type 'Contract Law Update', "
    "change description 'New state legislation limits indemnification scope in commercial contracts', "
    "impact assessment 'favorable for our client position, strengthens negotiation stance', "
    "client implications 'can push for narrower indemnification terms with legal backing'.",
    memory=memory_q2
)
print("🌍 Q2 Regulatory Update:", response)

# 규제 변경에 따른 전략 업데이트
response = await agent_q2.run(
    "Update legal strategy for 'CASE-001': strategy type 'Regulatory Leverage', "
    "approach 'cite new state legislation to support narrow indemnification position, strengthen negotiation leverage', confidence level 'high'.",
    memory=memory_q2
)
print("⚖️ Q2 Strategy Update:", response)

In [ ]:
# Q2 사건 진행 상황 추적
response = await agent_q2.run(
    "Track legal metrics for 'CASE-001': metric type 'Negotiation Progress', value 'Favorable terms secured', "
    "case_id 'CASE-001', quarter 'Q2 2024'.",
    memory=memory_q2
)
print("📈 Q2 Progress:", response)

# 전략 비교 테스트
response = await agent_q2.run(
    "How did the regulatory change impact CASE-001's strategy? Compare Q1 vs Q2 approaches.",
    memory=memory_q2
)
print("📊 Q2 Strategy Analysis:")
print(response)
print("\n✅ Expected: Q1 contract negotiation → Q2 regulatory leverage, strengthened position")

## 단계 7: Q3 법률 세션 - 사건 해결 및 새로운 사안

사건 해결 및 새로운 사건 접수로 진행합니다:

In [ ]:
# === Q3 법률 세션 ===
print("\n🗓️ === Q3: CASE RESOLUTION AND NEW MATTER ===")

agent_q3, memory_q3 = create_legal_session("q3")

# 사건 결과 기록
response = await agent_q3.run(
    "Log case outcome for 'CASE-001' with outcome type 'Settlement Agreement', "
    "result 'Favorable terms achieved - narrow indemnification scope, liability caps at $500K, mutual structure', "
    "lessons learned 'regulatory leverage was decisive, early precedent research paid off, client saved estimated $2M in potential liability'.",
    memory=memory_q3
)
print("📅 Q3 Case Resolution:", response)

# 새로운 사건 분석 시작
response = await agent_q3.run(
    "Analyze contract clause for 'CASE-002': clause type 'Non-Compete', "
    "risk level 'Medium', precedent reference 'TechCorp v. StartupInc (2020) - geographic and temporal scope must be reasonable'.",
    memory=memory_q3
)
print("💭 Q3 New Case Analysis:", response)

In [ ]:
# 포괄적인 법률 이력 회상 테스트
response = await agent_q3.run(
    "What is the complete legal analysis history? Include all cases, strategies, "
    "regulatory changes, and outcomes.",
    memory=memory_q3
)
print("📋 Q3 Complete History:")
print(response)
print("\n✅ Expected: CASE-001 journey → CASE-002 start, regulatory updates, strategy evolution")
# 주요 법률 발견 사항 명시적으로 추적
await agent_q3.run(
    "Save legal finding: finding 'Contract contains 3 high-risk clauses', confidence 'high'.",
    memory=memory_q3
)


In [ ]:
# 법률 메모리 추출을 위한 대기 시간
import asyncio
print("\n⏳ Waiting for legal memory extraction...")
await asyncio.sleep(90)
print("✅ Legal memory processing complete")


## 단계 8: Q4 법률 세션 - 연말 검토 및 계획

의미론적 검색 및 연간 법률 분석을 테스트합니다:

In [ ]:
# === Q4 법률 세션 ===
print("\n🗓️ === Q4: YEAR-END REVIEW AND PLANNING ===")

agent_q4, memory_q4 = create_legal_session("q4")

# 연간 법률 지표 추적
response = await agent_q4.run(
    "Track legal metrics: metric type '2024 Annual Performance', value 'Cases resolved: 2, Success rate: 100%, Client savings: $2.5M', "
    "case_id 'ANNUAL-SUMMARY', quarter '2024 Annual'.",
    memory=memory_q4
)
print("📈 Q4 Annual Metrics:", response)

# 규제 영향 상관관계 테스트
response = await agent_q4.run(
    "What regulatory changes have I documented this year? How did they impact case strategies?",
    memory=memory_q4
)
print("🌍 Q4 Regulatory Impact Analysis:")
print(response)
print("\n✅ Expected: Contract law update → strengthened CASE-001 negotiation position")

In [ ]:
# 유사한 법률 전략에 대한 의미론적 검색 테스트
response = await agent_q4.run(
    "What legal strategies have I used? Which were most effective based on case outcomes?",
    memory=memory_q4
)
print("⚖️ Q4 Strategy Effectiveness Analysis:")
print(response)
print("\n✅ Expected: Contract negotiation + regulatory leverage = successful outcomes")

## 단계 9: 2년차 Q1 세션 - 다년간 법률 관점

장기 법률 지식 및 실무 진화를 테스트합니다:

In [ ]:
# === 2년차 Q1 법률 세션 ===
print("\n🗓️ === YEAR 2 Q1: MULTI-YEAR LEGAL PERSPECTIVE ===")

agent_y2q1, memory_y2q1 = create_legal_session("year2-q1")

# 다년간 법률 실무 분석
response = await agent_y2q1.run(
    "Analyze my legal practice evolution: How have my cases and strategies developed over the past year? "
    "What were the key legal decisions and their outcomes?",
    memory=memory_y2q1
)
print("📊 Year 2 Q1 Practice Analysis:")
print(response)
print("\n✅ Expected: CASE-001 → CASE-002 progression, regulatory adaptation, strategy refinement")

In [ ]:
# 법률 판례 진화 추적 테스트
response = await agent_y2q1.run(
    "How have my legal precedents and strategies evolved? What case law have I applied and why?",
    memory=memory_y2q1
)
print("💭 Year 2 Q1 Precedent Evolution:")
print(response)
print("\n✅ Expected: Started with indemnification precedents → expanded to non-compete law")

## 단계 10: 최종 법률 실무 평가

장기 법률 분석 기능에 대한 포괄적인 테스트:

In [ ]:
# 최종 포괄적인 법률 실무 쿼리
response = await agent_y2q1.run(
    "Provide my complete legal practice portfolio: all cases with their legal journeys, "
    "strategy effectiveness, regulatory changes applied, precedent utilization, and case outcomes. "
    "Include lessons learned and best practices developed.",
    memory=memory_y2q1
)
print("💼 Complete Legal Practice Portfolio:")
print(response)
print("\n✅ Expected: Full case portfolio with strategy evolution, regulatory adaptation, and outcome analysis")

## 🧪 자동화된 테스트 검증
메모리 통합이 올바르게 작동하는지 검증하기 위해 이 셀을 실행하세요:

In [ ]:
# 검증 함수를 인라인으로 정의
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """agent가 세션 초반의 정보를 회상할 수 있는지 확인"""
        # 실질적인 응답 확인 (단순히 "모르겠습니다"가 아닌)
        has_content = len(response) > 50
        # 메모리 지표 확인
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """agent가 세션 내에서 컨텍스트를 유지하는지 확인"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """agent가 현재 쿼리를 이전 컨텍스트와 연결할 수 있는지 확인"""
        # 연결 언어 찾기
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# 모든 검증 테스트 실행
test_results = {}

# 테스트 1: 메모리 회상 - agent가 논의된 내용을 회상할 수 있는가?
response1 = await agent_y2q1.run("What have we discussed so far in this session?", memory=memory_y2q1)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"Response 1 length: {len(str(response1))} chars")

# 테스트 2: 세션 메모리 - agent가 컨텍스트를 유지하는가?
response2 = await agent_y2q1.run("What did we talk about earlier?", memory=memory_y2q1)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"Response 2 length: {len(str(response2))} chars")

# 테스트 3: 교차 참조 기능 - 이전 컨텍스트와 연결할 수 있는가?
response3 = await agent_y2q1.run("How does this relate to what we discussed before?", memory=memory_y2q1)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"Response 3 length: {len(str(response3))} chars")

# 결과 표시
validator.run_validation_summary(test_results)

## 요약

이 노트북에서는 다음을 시연했습니다:

✅ **장기 메모리 통합**: 세션 간 법률 분석을 위해 LlamaIndex와 함께 AgentCore Memory 사용

✅ **법률 사건 추적**: 여러 분기에 걸친 사건 진화 및 전략 개발

✅ **규제 인텔리전스**: 규제 변경 사항 및 사건 적용에 대한 의미론적 검색

✅ **법률 전략 진화**: 초기 분석에서 규제 적응 접근 방식으로의 자연스러운 진행

✅ **판례 관리**: 판례법 및 전략적 적용에 대한 상세한 추적

✅ **법률 실무 우수성**: 시간이 지남에 따른 포괄적인 사건 관리 및 결과 최적화

법률 문서 분석기는 장기 메모리가 분석기를 시간이 지남에 따라 더 똑똑해지는 지속적인 법률 파트너로 변환하여 완전한 사건 이력을 유지하고 확장된 실무 기간에 걸쳐 정교한 법률 지식 검색을 가능하게 하는 방법을 보여줍니다.

## 정리

이 노트북에서 사용된 리소스를 정리하기 위해 메모리를 삭제하겠습니다:

**참고**: 메모리를 영구적으로 삭제하려는 경우에만 실행하세요. memory_id 변수는 이 노트북의 앞부분에서 생성된 메모리의 ID를 포함해야 합니다.

In [ ]:
# AgentCore Memory 리소스 정리
try:
    from bedrock_agentcore.memory import MemoryClient
    
    client = MemoryClient(region_name=region)
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
    
except NameError as e:
    print(f"⚠️  Variable not defined: {e}")
    print("Run the notebook from the beginning or set variables manually:")
    print("# memory_id = 'your-memory-id-here'")
    print("# region = 'us-east-1'")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")